In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1, bias=False)      # Input 1, No. of Kernels 8, Kernel Size 3*3, Bias = False
        self.BN1 = nn.BatchNorm2d(8)                                #Performing BatchNormalization of 8 Output Channels
        nn.Dropout2d(0.25)                                          #Dropout(0.25)

        self.conv2 = nn.Conv2d(8, 16, 3, padding=1, bias=False)     #Input 8, No. of Kernels 16, Kernel Size 3*3, Bias = False
        self.BN2 =nn.BatchNorm2d(16)                                #Performing BatchNormalization of 16 Output Channels
        nn.Dropout2d(0.25)                                          #Dropout(0.25)

        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(16, 16, 3, padding=1,bias=False)     #Input 16, No. of Kernels 16, Kernel Size 3*3, Bias = False
        self.BN3 =nn.BatchNorm2d(16)                                #Performing BatchNormalization of 16 Output Channels
        nn.Dropout2d(0.25)                                          #Dropout(0.25)    

        self.conv4 = nn.Conv2d(16, 24, 3, padding=1,bias=False)     #Input 16, No. of Kernels 24, Kernel Size 3*3, Bias = False
        self.BN4 =nn.BatchNorm2d(24)                                #Performing BatchNormalization of 24 Output Channels
        self.D2 = nn.Dropout2d(0.25)                                #Added Dropout(0.25) after 4 Convolutions

        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(24, 32, 3, padding=1,bias=False)     #Input 24, No. of Kernels 32, Kernel Size 3*3, Bias = False
                          
        self.BN5 =nn.BatchNorm2d(32)                                #Performing BatchNormalization of 32 Output Channels
        nn.Dropout2d(0.25)                                          #Dropout(0.25)

        self.conv6 = nn.Conv2d(32, 10, 1,bias=False)                #Input 32, No. of Kernels 10, Kernel Size 1*1, Bias = False (Flatten)
        self.conv7 = nn.Conv2d(10, 10, 7,bias=False)                #Input 10, No. of Kernels 10, GAP of 7 convolution layers 7, Bias = False

    def forward(self, x):
        x = self.pool1(F.relu(self.BN2(self.conv2(F.relu(self.BN1(self.conv1(x)))))))
        x = self.pool2(F.relu(self.D2(self.BN4(self.conv4(F.relu(self.BN3(self.conv3(x))))))))
        x = F.relu(self.conv6(F.relu(self.BN5(self.conv5(x)))))
        x = (self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,152
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]           2,304
       BatchNorm2d-7           [-1, 16, 14, 14]              32
            Conv2d-8           [-1, 24, 14, 14]           3,456
       BatchNorm2d-9           [-1, 24, 14, 14]              48
        Dropout2d-10           [-1, 24, 14, 14]               0
        MaxPool2d-11             [-1, 24, 7, 7]               0
           Conv2d-12             [-1, 32, 7, 7]           6,912
      BatchNorm2d-13             [-1, 32, 7, 7]              64
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

In [0]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print("****************************************epoch{}".format(epoch))
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

****************************************epoch1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0744, Accuracy: 9749/10000 (97.49%)

****************************************epoch2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0169, Accuracy: 9948/10000 (99.48%)

****************************************epoch18


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0287, Accuracy: 9909/10000 (99.09%)

****************************************epoch19


loss=0.0007527768611907959 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 36.72it/s]



Test set: Average loss: 0.0202, Accuracy: 9937/10000 (99.37%)



Submitted as Group Actvity (G Uday Kiran, V Srilakshmi)